In [1]:
import uuid, mlflow, optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

UNIQUE_PREFIX = str(uuid.uuid4())[:8]

X = load_iris().data
y = load_iris().target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\pydantic\_internal\_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\pydantic\_internal\_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def objective(trial):
    params = {'max_depth': trial.suggest_int('max_depth', 2, 8)
              ,'random_state': trial.suggest_int('random_state',42, 123)}
    run_name = f'{UNIQUE_PREFIX}rfc'
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(params)
        clf = RandomForestClassifier(**params).fit(X_train, y_train)
        score = cross_val_score(clf, X_test, y_test, cv=3, scoring='accuracy').mean()
        mlflow.log_metric('accuracy', score)
    return score

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
if __name__ == '__main__':
    experiment_name = 'RandomForest'
    mlflow.set_tracking_uri('http://127.0.0.1:8081')
    mlflow.set_experiment(experiment_name)
    study = optuna.create_study(direction='maximize')
    # study.optimize(lambda trial: objective(trial), n_trials=5)
    study.optimize(objective, n_trials=5)
    best_param = study.best_params

    clf = RandomForestClassifier(**best_param).fit(X_train, y_train)
    score = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy').mean()
    mlflow.log_metric('accuracy', score)
    mlflow.sklearn.log_model(clf, "model")

[I 2023-11-14 23:26:23,395] A new study created in memory with name: no-name-32c857bb-7615-4478-884c-2cfe025968a9


[I 2023-11-14 23:26:23,925] Trial 0 finished with value: 1.0 and parameters: {'max_depth': 5, 'random_state': 57}. Best is trial 0 with value: 1.0.
[I 2023-11-14 23:26:24,491] Trial 1 finished with value: 1.0 and parameters: {'max_depth': 2, 'random_state': 55}. Best is trial 0 with value: 1.0.
[I 2023-11-14 23:26:25,044] Trial 2 finished with value: 1.0 and parameters: {'max_depth': 5, 'random_state': 123}. Best is trial 0 with value: 1.0.
[I 2023-11-14 23:26:25,568] Trial 3 finished with value: 1.0 and parameters: {'max_depth': 5, 'random_state': 102}. Best is trial 0 with value: 1.0.
[I 2023-11-14 23:26:26,091] Trial 4 finished with value: 1.0 and parameters: {'max_depth': 2, 'random_state': 114}. Best is trial 0 with value: 1.0.


In [5]:
mlflow.search_runs()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.max_depth,params.random_state,tags.mlflow.source.git.commit,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.source.type,tags.mlflow.user
0,92d1fc1644dd47d889449a830e448ae6,798764525771543735,RUNNING,mlflow-artifacts:/798764525771543735/92d1fc164...,2023-11-14 14:26:26.532000+00:00,NaT,0.92381,None,None,943bc617cd6d53c34a916b64e933c5d1b85cc1bf,"[{""run_id"": ""92d1fc1644dd47d889449a830e448ae6""...",c:\Users\jw160\OneDrive\test\bento2\.venv_bent...,unleashed-croc-419,LOCAL,jw160
1,00a8df71b7c44ebe9dd7fb01ecff7a55,798764525771543735,FINISHED,mlflow-artifacts:/798764525771543735/00a8df71b...,2023-11-14 14:26:25.578000+00:00,2023-11-14 14:26:26.078000+00:00,1.00000,2,114,943bc617cd6d53c34a916b64e933c5d1b85cc1bf,None,c:\Users\jw160\OneDrive\test\bento2\.venv_bent...,3fce220erfc,LOCAL,jw160
2,ca038231c2254e3bb12bbe9145bf385c,798764525771543735,FINISHED,mlflow-artifacts:/798764525771543735/ca038231c...,2023-11-14 14:26:25.052000+00:00,2023-11-14 14:26:25.556000+00:00,1.00000,5,102,943bc617cd6d53c34a916b64e933c5d1b85cc1bf,None,c:\Users\jw160\OneDrive\test\bento2\.venv_bent...,3fce220erfc,LOCAL,jw160
3,2c7fda558f864095a8fde045862dfac8,798764525771543735,FINISHED,mlflow-artifacts:/798764525771543735/2c7fda558...,2023-11-14 14:26:24.501000+00:00,2023-11-14 14:26:25.031000+00:00,1.00000,5,123,943bc617cd6d53c34a916b64e933c5d1b85cc1bf,None,c:\Users\jw160\OneDrive\test\bento2\.venv_bent...,3fce220erfc,LOCAL,jw160
4,0b46dcbeb8b049d5bab48e8c48ce2c74,798764525771543735,FINISHED,mlflow-artifacts:/798764525771543735/0b46dcbeb...,2023-11-14 14:26:23.934000+00:00,2023-11-14 14:26:24.479000+00:00,1.00000,2,55,943bc617cd6d53c34a916b64e933c5d1b85cc1bf,None,c:\Users\jw160\OneDrive\test\bento2\.venv_bent...,3fce220erfc,LOCAL,jw160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1ad9419e2bae48f0a256341954200d81,798764525771543735,FINISHED,mlflow-artifacts:/798764525771543735/1ad9419e2...,2023-11-13 11:30:33.840000+00:00,2023-11-13 11:30:34.335000+00:00,1.00000,3,83,None,None,model.py,66a01531rfc,LOCAL,jw160
65,97bac35208a6461cb1c5b1be5ae21990,798764525771543735,FINISHED,mlflow-artifacts:/798764525771543735/97bac3520...,2023-11-13 11:30:33.349000+00:00,2023-11-13 11:30:33.815000+00:00,1.00000,5,92,None,None,model.py,66a01531rfc,LOCAL,jw160
66,315337ac463b406189612ee012da26b1,798764525771543735,FINISHED,mlflow-artifacts:/798764525771543735/315337ac4...,2023-11-13 11:30:32.850000+00:00,2023-11-13 11:30:33.323000+00:00,1.00000,5,104,None,None,model.py,66a01531rfc,LOCAL,jw160
67,7cdce81be7524736b795bb9c37c3da6f,798764525771543735,FINISHED,mlflow-artifacts:/798764525771543735/7cdce81be...,2023-11-13 11:30:32.349000+00:00,2023-11-13 11:30:32.824000+00:00,1.00000,4,44,None,None,model.py,66a01531rfc,LOCAL,jw160
